# Multiple input and output neurons

In the [previous chapter](./1.1%20Eval%20-%20basic%20eval.ipynb) we constructed
a SQL query for a basic neural network that only had a single input and output
neuron. We'll now adapt this query to be able to handle multiple input and/or
output neurons. The network can have any number of hidden layers, but we
assume we know how many of them beforehand.

## Creating the model

Let's start of by creating and training a network that accepts two input values,
and predicts two output values.

In [12]:
import torch
import utils.sqlite as db
import utils.nn as nn
import pandas as pd

torch.manual_seed(223)

# The function we're going to learn accepts 2 input values and returns 2 output
# values.
def f(x, y):
    return [2*x, 4*y]

num_samples = 100
x_train = torch.randn(num_samples, 2) * 100
y_train = [f(x,y) for [x,y] in x_train]

model = nn.ReLUFNN(input_size=2, output_size=2, hidden_size=2, num_hidden_layers=1)
nn.train(model, x_train, y_train, save_path="models/eval_multiple_io.pt")

And let's load the trained model into the database again.

In [13]:
db.load_pytorch_model_into_db(model)

## Constructing the query

Let's create the eval query. We first give it in full again, and then
go over each part separately.

```sql
WITH input_values AS (
    SELECT 1 as input_node_idx, ? as input_value 
    UNION 
    SELECT 2 as input_node_idx, ? as input_value
),
input_nodes AS (
    SELECT
        id,
        bias,
        ROW_NUMBER() OVER (ORDER BY id) AS input_node_idx
    FROM node
    WHERE id NOT IN
    (
        SELECT dst FROM edge
    )
),
t1 AS (
    SELECT
        MAX(
            0,
            n.bias + SUM(e.weight * v.input_value)
        ) AS t1,
        e.dst AS id
    FROM edge e
    JOIN input_nodes i ON i.id = e.src
    JOIN node n ON e.dst = n.id
    JOIN input_values v ON i.input_node_idx = v.input_node_idx
    GROUP BY e.dst, n.bias
),
outputs AS (
    SELECT
        n.bias + SUM(e.weight * t1.t1) AS output_value,
        e.dst AS output_node_id
    FROM edge e
    JOIN t1 ON t1.id = e.src
    JOIN node n ON e.dst = n.id
    GROUP BY e.dst, n.bias
)
SELECT * FROM outputs ORDER BY output_node_id;
```

Note that the bulk of the query is similar to [the one from the first
chapter](./3.1.%20generic%20eval.ipynb). Let's go over the differences one by
one.

```sql
WITH input_values AS (
    SELECT 1 as input_node_idx, ? as input_value 
    UNION 
    SELECT 2 as input_node_idx, ? as input_value
)
```

Now that we have multiple inputs, we manually pass them as a union of hardcoded
values. We also explicitly state which input neuron each value belongs to. Note
that it wouldn't be very difficult to instead read input values from an existing
table.

```sql
input_nodes AS (
    SELECT
        id,
        bias,
        ROW_NUMBER() OVER (ORDER BY id) AS input_node_idx
    FROM node
    WHERE id NOT IN
    (
        SELECT dst FROM edge
    )
)
```

We still select the input nodes by checking which nodes do not have an incoming
edge. However, we also use SQL's `ROW_NUMBER` functionality to assign a numeric
index to each input node. This way, we can match them to the corresponding input
value. This is done by the following join in the $t_1$ term:

```sql
JOIN input_values v ON i.input_node_idx = v.input_node_idx
```

The output CTE is identical, with one addition: we add an explicit `ORDER
BY`-clause so that the output is always in the expected order.

The following python code constructs a query for a given neural network,
dynamically creating the input statements and inductive terms for each hidden
layer.

In [14]:
def eval_nn(model, input_value):
    num_layers = int(len(model.state_dict()) / 2)

    # Add as many input clauses as needed.
    input_clauses = []
    for i,_ in enumerate(input_value):
        input_clauses.append(f"SELECT {i + 1} as input_node_idx, ? as input_value")

    query = f"""
        WITH input_values AS (
            {" UNION ".join(input_clauses)}
        ),
        input_nodes AS (
            SELECT
                id,
                bias,
                ROW_NUMBER() OVER (ORDER BY id) AS input_node_idx
            FROM node
            WHERE id NOT IN
            (
                SELECT dst FROM edge
            )
        ),
        t1 AS (
            SELECT
                MAX(
                    0,
                    n.bias + SUM(e.weight * v.input_value)
                ) AS t1,
                e.dst AS id
            FROM edge e
            JOIN input_nodes i ON i.id = e.src
            JOIN node n ON e.dst = n.id
            JOIN input_values v ON i.input_node_idx = v.input_node_idx
            GROUP BY e.dst, n.bias
        ),
        """

    # Everything else is the same as in the previous part.
    for hidden_layer in range(2, num_layers):
        curr = hidden_layer
        prev = hidden_layer - 1
        query += f"""
            t{curr} AS (
                SELECT
                    MAX(
                        0,
                        n.bias + SUM(e.weight * t{prev}.t{prev})
                    ) AS t{curr},
                    e.dst AS id
                FROM edge e
                JOIN t{prev} ON t{prev}.id = e.src
                JOIN node n ON e.dst = n.id
                GROUP BY e.dst, n.bias
            ),
        """

    prev = num_layers - 1
    query += f"""
        outputs AS (
            SELECT
                n.bias + SUM(e.weight * t{prev}.t{prev}) AS output_value,
                e.dst AS output_node_id
            FROM edge e
            JOIN t{prev} ON t{prev}.id = e.src
            JOIN node n ON e.dst = n.id
            GROUP BY e.dst, n.bias
        )
        SELECT * FROM outputs ORDER BY output_node_id;
    """

    results = db.con.execute(query, input_value).fetchall()

    return [result[0] for result in results]

Let's run our query on the model we created and compare the output.

In [15]:
nn_output = model(torch.tensor([5,20], dtype=torch.float32)).detach().numpy()
sql_output = eval_nn(model, [5, 20])

print(f"The neural network predicted {nn_output}")
print(f"The SQL query calculated {sql_output}")

The neural network predicted [-24.508732  54.89358 ]
The SQL query calculated [-24.508734581785873, 54.89358546175163]


And let's do the same thing for a larger amount of input samples.

In [16]:
df = pd.DataFrame(columns=['in_1', 'in_2', 'nn_out_1', 'nn_out_2', 'sql_out_1', 'sql_out_2'])

for i, input_vals in enumerate(x_train):
    [nn_out_1, nn_out_2] = model(input_vals).detach().numpy()
    [sql_out_1, sql_out_2] = eval_nn(model, input_vals.tolist())
    df.loc[i] = [
        input_vals[0].item(), input_vals[1].item(),
        nn_out_1, nn_out_2,
        sql_out_1, sql_out_2
    ]

delta_1_avg = (abs(df['nn_out_1'] - df['sql_out_1'])).mean()
delta_2_avg = (abs(df['nn_out_2'] - df['sql_out_2'])).mean()

print(f"The average difference for output value 1 is {delta_1_avg}")
print(f"The average difference for output value 2 is {delta_2_avg}")

The average difference for output value 1 is 3.84342975134011e-06
The average difference for output value 2 is 1.4083713261818786e-05


We can see that the outputs are again identical, barring some floating point
errors.

## Adding more hidden layers

We now have a generic `eval` that works with multiple input/output neurons.
Let's also check how it performs with more hidden layers.

In [17]:
bigger_model = nn.ReLUFNN(input_size=2, output_size=2, hidden_size=5, num_hidden_layers=50)
nn.train(bigger_model, x_train, y_train, save_path="models/eval_multiple_io_bigger.pt")
db.load_pytorch_model_into_db(bigger_model)

In [18]:
nn_output = bigger_model(torch.tensor([5,20], dtype=torch.float32)).detach().numpy()
sql_output = eval_nn(bigger_model, [5, 20])

print(nn_output)
print(sql_output)

[-33.27764  -32.772896]
[-33.27763928976887, -32.77289402078124]


In [19]:
df = pd.DataFrame(columns=['in_1', 'in_2', 'nn_out_1', 'nn_out_2', 'sql_out_1', 'sql_out_2'])

for i, input_vals in enumerate(x_train):
    [nn_out_1, nn_out_2] = bigger_model(input_vals).detach().numpy()
    [sql_out_1, sql_out_2] = eval_nn(bigger_model, input_vals.tolist())
    df.loc[i] = [
        input_vals[0].item(), input_vals[1].item(),
        nn_out_1, nn_out_2,
        sql_out_1, sql_out_2
    ]

delta_1_avg = (abs(df['nn_out_1'] - df['sql_out_1'])).mean()
delta_2_avg = (abs(df['nn_out_2'] - df['sql_out_2'])).mean()

print(f"The average difference for output value 1 is {delta_1_avg}")
print(f"The average difference for output value 2 is {delta_2_avg}")

The average difference for output value 1 is 2.006617847882808e-06
The average difference for output value 2 is 1.7922070441045435e-06


Again, our query gives near identical result. 

## Conclusion

Where we first only allowed one input and one output neuron, we can now evaluate
neural networks with any number of input and output neurons. In the next
chapter, we'll go one step further and add the possibility to evaluate multiple
input sets at the same time.